# APOBEC mutagenesis in different genome regions
## Table of contents <a name="TOC"></a>
1. [APOBEC mutagenesis in R-loops](#RL)
    1. [Downloading coordinates of R-loops and hg38](#input_data)
    2. [Searching for potential APOBEC targets (TC)](#searching_motifs)
    3. [Searching for APOBEC-mutated targets and calculation D_APOBEC](#D_APOBEC_RL)
2. [APOBEC mutagenesis in transposons](#TE)
    1. [Downloading RepeatMasker database](#RM)
    2. [Searching for potential APOBEC targets (TC) in and outside TE](#searching_motifs_TE)
    3. [Searching for APOBEC-mutated targets in and outside TE and calculation D_APOBEC](#D_APOBEC_TE)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import numpy as np
import gzip
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [ ]:
import re
import random
import gzip
from natsort import index_natsorted

## APOBEC mutagenesis in R-loops
<a name='RL'></a>

### hg38

<a name='input_data'></a>
[Return to Table of Contents](#TOC)

downloading hg38 genome https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/

In [ ]:
chr_list = list(range(1,23))

hg38_genome = dict()
with gzip.open("genome/hg38.fa.gz", "rt") as fasta_file:
    for sequence in SimpleFastaParser(fasta_file):
        chrom = sequence[0][3:]
        if chrom in [str(chrom) for chrom in chr_list]:
            hg38_genome[int(chrom)] = sequence[1].upper()
            print(chrom, end=', ')
        elif chrom in ['X', 'Y']:
            hg38_genome[chrom] = sequence[1].upper()
            print(chrom, end=', ')

### R-loops coordinates

[Return to Table of Contents](#TOC)

source https://academic.oup.com/nar/article/50/13/7260/6618547#366685035

In [ ]:
RL_regions = pd.read_csv('RL_regions.csv')
RL_regions.head()

### Searching for potential APOBEC targets (TC)

<a name='searching_motifs'></a>
[Return to Table of Contents](#TOC)

In [ ]:
sequences = []
RL_regions = RL_regions[RL_regions['chrom'] != 'chrM']
for row in range(0, len(RL_regions)):
    N_chr = RL_regions.iloc[row]['chrom'][3:]
    if N_chr != 'X' and N_chr != 'Y':
        N_chr = int(N_chr)
    start = RL_regions.iloc[row]['start']
    end = RL_regions.iloc[row]['end']
    sequences.append(hg38_genome[N_chr][start:end])
RL_regions['seq'] = sequences

In [ ]:
RL_regions.head()

In [ ]:
RL_regions['N_motifs'] = RL_regions.seq.str.count('TC') + RL_regions.seq.str.count('GA')
RL_regions.head()

In [ ]:
print('Number of potential APOBEC targets in R-loops: ', RL_regions.N_motifs.sum())

In [ ]:
chrom_list_XY = list(range(1,23)) + ['X', 'Y']
motifs_count = dict()
for chrom in chrom_list_XY:
    motifs_count[chrom] = pd.Series([hg38_genome[chrom].count('TC'), hg38_genome[chrom].count('GA')], index = ['TC', 'GA'])
motifs_count = pd.concat(motifs_count).unstack()

In [ ]:
motifs_count['TC_GA'] = motifs_count['TC']+motifs_count['GA']
motifs_count

In [ ]:
motifs_count.sum()

In [ ]:
print('Number of potential APOBEC targets NOT in R-loops: ', 350769311-16990012)

### Searching for APOBEC-mutated targets and calculation the mutation density D_APOBEC

<a name='D_APOBEC_RL'></a>
[Return to Table of Contents](#TOC)

In [ ]:
cancer = ['BLCA', 'BRCA', 'CESC', 'HNSC', 'LUAD', 'LUSC']

In [ ]:
df = pd.read_csv('APOBECflag/snv_mnv_BLCA-US_short.tsv', sep='\t')
df.head()

In [ ]:
#translate to hg38
#make bed file
# 1-based coordinates
for cancer_type in cancer:
    df = pd.read_csv('APOBECflag/snv_mnv_'+cancer_type+'-US_short.tsv', sep='\t', dtype={'Chr': 'string'})
    df = df[df["APOBECrelaxed"]==1]
    df['chrom'] = 'chr' + df.Chr
    df['start'] = df.Position - 1
    df[['chrom', 'start', 'Position', 'Sample']].to_csv('APOBECflag/'+cancer_type+'.bed', index=False, header=None, sep='\t')

#### Python script

the program below was run separately on the command line as a python script

In [ ]:
import sys
import os

import pandas as pd
import numpy as np


# input data
RL_path = '.../RL_regions.csv'
cancer = ['BLCA', 'BRCA', 'CESC', 'HNSC', 'LUAD', 'LUSC']
cancer_path = 'APOBECflag/'
outfile_path = ''
# Number of potential APOBEC targets in R-loops
N_TCN_RL = 16990012
# Number of potential APOBEC targets outside R-loops
N_TCN_out = 333779299

# Searching for position that are in R-loops

def closest_value(input_list, input_value):
    arr = np.asarray(input_list)
    i = (np.abs(arr - input_value)).argmin()
    return i

RL_regions = pd.read_csv(RL_path)

snv_mnv = dict()
chrom_list_XY = list(range(1,23)) + ['X', 'Y']
chrom_list_XY = ['chr'+str(i) for i in chrom_list_XY]

for cancer_type in cancer:
    print(cancer_type, end=', ')
    df = pd.read_csv(cancer_path+cancer_type+'_hg38.bed', sep='\t', names=['chr', 'start', 'end', 'sample'])
    df = df[df['chr'].isin(chrom_list_XY)]
    df['inRloop'] = 0
    
    print(len(df))
    
    df = df.reset_index(drop=True)
    for row in range(0, len(df)):
        
        Chr = df.iloc[row]['chr']
        position = df.iloc[row]['start']
        
        RL_regions_chr = RL_regions[RL_regions["chrom"]==Chr]
        RL_regions_chr = RL_regions_chr.reset_index(drop=True)
        
        # searching of row's number with the closest to position value
        if __name__ == "__main__" :
            N = closest_value(RL_regions_chr.start.tolist(),position)
        
        # position may be in this row (N) or in previous row (N-1)
        RL_coordinates = [[RL_regions_chr.iloc[N-1]['start'], RL_regions_chr.iloc[N-1]['end']], [RL_regions_chr.iloc[N]['start'], RL_regions_chr.iloc[N]['end']]]
        
        for x in RL_coordinates:
            if position <= x[1] and position >= x[0]:
                df.at[row, 'inRloop'] = 1
                break
    snv_mnv[cancer_type] = df
    

# Calculation the mutation density D_APOBEC of the APOBEC mutagenesis 

APOBEC_mutagenesis = pd.DataFrame(columns=['cancer','sample','APOBEC_mutagenesis_inRloops','APOBEC_mutagenesis_outsideRloops'])
for cancer_type in cancer:
    df = snv_mnv[cancer_type]
    for sample in df['sample'].unique():
        df_sample = df[df['sample']==sample]

        # number of mutated targets inside R loops
        N_APOBEC_RL = df_sample.inRloop.sum()
        
        # number of mutated targets outside R loops
        N_APOBEC_out = len(df_sample) - N_APOBEC_RL
        
        D_APOBEC_RL = N_APOBEC_RL / N_TCN_RL
        D_APOBEC_out = N_APOBEC_out / N_TCN_out
        
        APOBEC_mutagenesis.loc[len(APOBEC_mutagenesis.index)] = [cancer_type, sample, D_APOBEC_RL, D_APOBEC_out]

APOBEC_mutagenesis.to_csv(outfile_path, sep='\t')

## APOBEC mutagenesis in transposons
<a name='TE'></a>

### Downloading RepeatMasker database
<a name='RM'></a>
[Return to Table of Contents](#TOC)

source: https://genome.ucsc.edu/cgi-bin/hgTables?hgsid=1725823364_aFa8Z0U0ldymizRxYJXFsANb2sal&clade=mammal&org=Human&db=hg38&hgta_group=rep&hgta_track=cons100way&hgta_table=0&hgta_regionType=genome&position=chr2%3A25%2C160%2C915-25%2C168%2C903&hgta_outputType=wigData&hgta_outFileName=RepeatMasker.gtf

repeatmasker database description: https://genome.ucsc.edu/cgi-bin/hgTables?db=hg38&hgta_group=rep&hgta_track=rmsk&hgta_table=rmsk&hgta_doSchema=describe+table+schema

In [ ]:
f = gzip.open("RepeatMasker_hg38.gtf.gz", mode='rb')
RM = pd.read_csv(f, sep = "\t")
RM.head()

In [ ]:
chrom_list_XY = list(range(1,23)) + ['X', 'Y']
chrom_list_XY = ['chr'+str(i) for i in chrom_list_XY]

RM = RM[RM['genoName'].isin(chrom_list_XY)]
RM.repClass.unique()

In [ ]:
RM = RM.replace('LTR?', 'LTR')
RM = RM.replace('RC?', 'RC')
RM = RM.replace('SINE?', 'SINE')
RM = RM.replace('DNA?', 'DNA')
RM = RM.replace('DNA', 'DNA_transposons')
RM = RM.replace('Retroposon', 'SVA')
RM = RM.replace('RC', 'RC_transposons')

TE = ['LINE', 'SINE', 'LTR', 'SVA', 'RC_transposons', 'DNA_transposons']
RM_transposons = RM[RM['repClass'].isin(TE)]

In [ ]:
RM_transposons.genoName.unique()

### Searching for potential APOBEC targets (TC) in and outside TE

<a name='searching_motifs_TE'></a>
[Return to Table of Contents](#TOC)

In [ ]:
# translate the coordinates to + strand

RM_transposons = RM_transposons.sort_values(by=['strand'])
RM_transposons = RM_transposons.reset_index(drop=True)

#number of row with the first '-' strand value
separator_strand = len(RM_transposons[RM_transposons['strand']=='+'])

transposons_coordinates = RM_transposons.copy()

for row in range(separator_strand, len(transposons_coordinates)):

    N_chr = transposons_coordinates.iloc[row]['genoName'][3:]
    if N_chr != 'X' and N_chr != 'Y':
        N_chr = int(N_chr)
    start = int(transposons_coordinates.iloc[row]['genoStart'])
    end = int(transposons_coordinates.iloc[row]['genoEnd'])
    
    #translate coordinates to coding strand
    chr_len = len(hg38_genome[N_chr])
    start1 = chr_len-end-1
    end1 = chr_len-start-1
    
    transposons_coordinates.at[row, 'genoStart'] = start1
    transposons_coordinates.at[row, 'genoEnd'] = end1
    
transposons_coordinates = transposons_coordinates[['genoName', 'genoStart', 'genoEnd', 'repName', 'repClass',
       'repFamily']]
transposons_coordinates.to_csv('transposons_coordinates.csv', sep='\t', index=False)

In [ ]:
RM_transposons.iloc[separator_strand-1]['genoStart']

In [ ]:
transposons_coordinates.iloc[separator_strand-1]['genoStart']

In [ ]:
RM_transposons.iloc[separator_strand]['genoStart']

In [ ]:
transposons_coordinates.iloc[separator_strand]['genoStart']

In [ ]:
transposons_coordinates['N_motifs'] = 0
print(len(transposons_coordinates))

for row in range(0, len(transposons_coordinates)):
    
    #cheking
    if row % 1000000 == 0:
        print("I'm working")

    N_chr = transposons_coordinates.iloc[row]['genoName'][3:]
    if N_chr != 'X' and N_chr != 'Y':
        N_chr = int(N_chr)
    start = int(transposons_coordinates.iloc[row]['genoStart'])
    end = int(transposons_coordinates.iloc[row]['genoEnd'])
    seq = hg38_genome[N_chr][start:end]
    transposons_coordinates.at[row, 'N_motifs'] = seq.count('TC') + seq.count('GA')
    
transposons_coordinates.head()

In [ ]:
df = transposons_coordinates[['repClass', 'N_motifs']]
df = df.groupby('repClass').sum()
df = df.rename(columns={"N_motifs": "N_motifs_inTE"})
df['N_motifs_outsideTE'] = 350769311 - df.N_motifs_inTE
df

In [ ]:
print(df.to_dict('index'))

### Searching for APOBEC-mutated targets in and outside TE and calculation D_APOBEC

<a name='D_APOBEC_TE'></a>
[Return to Table of Contents](#TOC)

In [12]:
transposons_coordinates = pd.read_csv(TE_path, sep='\t')
transposons_coordinates

,genoName,genoStart,genoEnd,repName,repClass,repFamily
0,chr1,67108753,67109046,L1P5,LINE,L1
1,chr14,33473772,33473956,MER58A,DNA_transposons,hAT-Charlie
2,chr14,33473220,33473364,MER5A,DNA_transposons,hAT-Charlie
3,chr14,33449304,33449513,MER58A,DNA_transposons,hAT-Charlie
4,chr14,33444302,33444707,L3,LINE,CR1
...,...,...,...,...,...,...
4516043,chr5,46705798,46706100,AluSc8,SINE,Alu
4516044,chr5,46705076,46705412,AluSx1,SINE,Alu
4516045,chr20,16960563,16960742,AluSx1,SINE,Alu
4516046,chr5,13374666,13374716,L1MB1,LINE,L1


In [14]:
chrom_list_XY = list(range(1,2)) + ['X', 'Y']
chr_list_XY = ['chr'+str(i) for i in chrom_list_XY]
transposons_coordinates = transposons_coordinates[transposons_coordinates['genoName'].isin(chr_list_XY)]
transposons_coordinates = transposons_coordinates[transposons_coordinates['repClass']== 'RC_transposons']
transposons_coordinates.reset_index(drop=True)

,genoName,genoStart,genoEnd,repName,repClass,repFamily
0,chr1,210480446,210480570,Helitron3Na_Mam,RC_transposons,Helitron
1,chr1,210480178,210480317,Helitron3Na_Mam,RC_transposons,Helitron
2,chr1,209749616,209749652,Helitron3Na_Mam,RC_transposons,Helitron
3,chr1,235711185,235711340,Helitron3Na_Mam,RC_transposons,Helitron
4,chr1,235710308,235710411,Helitron3Na_Mam,RC_transposons,Helitron
...,...,...,...,...,...,...
241,chr1,83697139,83697360,Helitron2Na_Mam,RC_transposons,Helitron?
242,chr1,218900078,218900237,Helitron2Na_Mam,RC_transposons,Helitron?
243,chr1,214560482,214560721,Helitron1Nb_Mam,RC_transposons,Helitron
244,chr1,214560750,214560983,Helitron1Na_Mam,RC_transposons,Helitron


#### Python script

the program below was run separately on the command line as a python script

In [17]:
import pandas as pd
import numpy as np
import gzip
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Seq import MutableSeq

# input data
genome_path = 'genome/hg38.fa.gz'
TE_path = 'transposons_coordinates.csv'

cancer = ['BLCA', 'BRCA', 'CESC', 'HNSC', 'LUAD', 'LUSC']
cancer_path = 'APOBECflag/'
outfile_path = ''

# number of the potential APOBEC targets in the genome
N_motifs_genome = 350769311


# read genome file
chr_list = list(range(1,23))
hg38_genome = dict()
with gzip.open(genome_path, "rt") as fasta_file:
    for sequence in SimpleFastaParser(fasta_file):
        chrom = sequence[0][3:]
        if chrom in [str(chrom) for chrom in chr_list]:
            hg38_genome[int(chrom)] = sequence[1].upper()
            print(chrom, end=', ')
        elif chrom in ['X', 'Y']:
            hg38_genome[chrom] = sequence[1].upper()
            print(chrom, end=', ')

# read file with transposon coordinates
transposons_coordinates = pd.read_csv(TE_path, sep='\t')


# count potential APOBEC targets (TC) in TE
transposons_coordinates['N_motifs'] = 0
print(len(transposons_coordinates))
for row in range(0, len(transposons_coordinates)):
    N_chr = transposons_coordinates.iloc[row]['genoName'][3:]
    if N_chr != 'X' and N_chr != 'Y':
        N_chr = int(N_chr)
    start = int(transposons_coordinates.iloc[row]['genoStart'])
    end = int(transposons_coordinates.iloc[row]['genoEnd'])
    seq = hg38_genome[N_chr][start:end]
    transposons_coordinates.at[row, 'N_motifs'] = seq.count('TC') + seq.count('GA')
N_motifs_TE = transposons_coordinates[['repClass', 'N_motifs']]
N_motifs_TE = N_motifs_TE.groupby('repClass').sum()
N_motifs_TE = N_motifs_TE.rename(columns={"N_motifs": "N_motifs_inTE"})
N_motifs_TE['N_motifs_outsideTE'] = N_motifs_genome - N_motifs_TE.N_motifs_inTE
    

# result dataframe with D_APOBEC
col_names = ['cancer','sample', 'transposon_class', 'N_motifs_inTE', 'N_motifs_outsideTE', 'N_mutated_motifs_inTE', 'N_mutated_motifs_outsideTE', 'D_APOBEC_in', 'D_APOBEC_outside']
APOBEC_mutagenesis_TE = pd.DataFrame(columns=col_names)


# count mutated targets in TE
# replace mutated position in the genome to 'N' by cancer type and sample
hg38_genome_mutated = hg38_genome.copy()
for key, value in hg38_genome_mutated.items():
    hg38_genome_mutated[key] = MutableSeq(value)

chrom_list_XY = list(range(1,23)) + ['X', 'Y']
chr_list_XY = ['chr'+str(i) for i in chrom_list_XY]

for cancer_type in cancer:
    print(cancer_type)
    df = pd.read_csv(cancer_path+cancer_type+'_hg38.bed', sep='\t', names=['chr', 'start', 'end', 'Sample'])
    df = df[df['chr'].isin(chr_list_XY)]
    
    for sample in df.Sample.unique():
        df2 = df[df['Sample']==sample]
        df2 = df2.reset_index(drop=True)
        
        mutated_sample_genome = hg38_genome_mutated.copy()
        not_mutated_motifs_count = dict()
    
        for Chr in chrom_list_XY:
            N_chr = 'chr'+str(Chr)
            df3 = df2[df2['chr']==N_chr]
            df3 = df3.reset_index(drop=True)
            mutation_list = df3.start.to_list()
            for ind in mutation_list:
                mutated_sample_genome[Chr][ind] = 'N'
                
            # count the number of NOT mutated motifs in the genome
            not_mutated_motifs_count[Chr] = pd.Series([mutated_sample_genome[Chr].count('TC'), mutated_sample_genome[Chr].count('GA')], index = ['TC', 'GA'])
        not_mutated_motifs_count = pd.concat(not_mutated_motifs_count).unstack()
        not_mutated_motifs_count['TC_GA'] = not_mutated_motifs_count['TC']+not_mutated_motifs_count['GA']
        N_mutated_motifs_genome = N_motifs_genome - not_mutated_motifs_count.TC_GA.sum()

        
        # count the number of NOT mutated motifs in TE for the sample
        transposons_coordinates['N_not_mutated_motifs'] = 0
        for row in range(0, len(transposons_coordinates)):
            N_chr = transposons_coordinates.iloc[row]['genoName'][3:]
            if N_chr != 'X' and N_chr != 'Y':
                N_chr = int(N_chr)
            start = int(transposons_coordinates.iloc[row]['genoStart'])
            end = int(transposons_coordinates.iloc[row]['genoEnd'])
            seq = mutated_sample_genome[N_chr][start:end]
            transposons_coordinates.at[row, 'N_not_mutated_motifs'] = seq.count('TC') + seq.count('GA')

        transposons_coordinates['N_mutated_motifs'] = transposons_coordinates.N_motifs - transposons_coordinates.N_not_mutated_motifs
        
        grouped_TE = transposons_coordinates[['repClass', 'N_mutated_motifs']]
        grouped_TE = grouped_TE.groupby('repClass').sum()
        grouped_TE = grouped_TE.rename(columns={'N_mutated_motifs':'N_mutated_motifs_inTE'})
        grouped_TE['N_mutated_motifs_outsideTE'] = N_mutated_motifs_genome - grouped_TE.N_mutated_motifs_inTE
        
        new_df = pd.concat([N_motifs_TE, grouped_TE], axis=1)
        new_df['D_APOBEC_in'] = new_df.N_mutated_motifs_inTE / new_df.N_motifs_inTE
        new_df['D_APOBEC_outside'] = new_df.N_mutated_motifs_outsideTE / new_df.N_motifs_outsideTE
        new_df = new_df.reset_index()
        new_df = new_df.rename(columns={'repClass':'transposon_class'})
        new_df = new_df[['transposon_class', 'N_motifs_inTE', 'N_motifs_outsideTE', 'N_mutated_motifs_inTE', 'N_mutated_motifs_outsideTE', 'D_APOBEC_in', 'D_APOBEC_outside']]
        for row in range(0, len(new_df)):
            APOBEC_mutagenesis_TE.loc[len(APOBEC_mutagenesis_TE.index)] = [cancer_type, sample]+new_df.iloc[row].to_list()

APOBEC_mutagenesis_TE.to_csv(outfile_path, sep='\t')

1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 3, 4, 5, 6, 7, 8, 9, X, Y, 246
BLCA
BRCA


KeyboardInterrupt: 